In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR


from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor




from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('gurgaon_properties_post_feature_selection_v2.csv')

In [3]:
df.sample(5)

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
1100,flat,sector 104,0.40,2.0,2.0,1,Relatively New,515.000000,0.0,0.0,0.0,Low,High Floor
1264,flat,sector 111,3.50,4.0,5.0,3+,Relatively New,2700.000000,1.0,0.0,1.0,Low,Mid Floor
1773,house,sector 9,3.50,9.0,9.0,3+,Relatively New,2610.000000,0.0,0.0,0.0,Low,Mid Floor
3039,flat,sohna road,0.78,2.0,2.0,3,Under Construction,1222.000000,0.0,0.0,0.0,Medium,Mid Floor
677,flat,sohna road,0.22,1.0,1.0,1,Relatively New,477.809521,0.0,0.0,1.0,Low,Mid Floor


In [7]:
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [95]:
df

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,unfurnished,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,unfurnished,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,semifurnished,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,unfurnished,High,Mid Floor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3549,flat,sector 84,0.37,2.0,2.0,1,Relatively New,532.0,0.0,0.0,unfurnished,Medium,Mid Floor
3550,house,sector 109,6.00,5.0,5.0,3+,Relatively New,6228.0,1.0,1.0,unfurnished,High,Low Floor
3551,flat,sector 2,0.60,1.0,1.0,1,Moderately Old,665.0,0.0,0.0,semifurnished,Medium,Mid Floor
3552,house,sector 43,15.50,5.0,6.0,3,Moderately Old,5490.0,1.0,1.0,unfurnished,Medium,Mid Floor


In [11]:
X = df.drop(columns=['price'])
y = df['price']

In [13]:
y_transformed = np.log1p(y)

## Target encoder


In [43]:
!pip install category_encoders



In [16]:
import category_encoders as ce

columns_to_encode = ['property_type', 'balcony', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [22]:
!pip install -U scikit-learn


In [18]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [20]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [22]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [24]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [26]:
model_df

,name,r2,mae
0,linear_reg,0.828997,0.714540
1,svr,0.863266,0.583070
2,ridge,0.829011,0.715103
3,LASSO,-0.003598,1.576392
4,decision tree,0.799118,0.583263
5,random forest,0.893453,0.472876
6,extra trees,0.893545,0.475069
7,gradient boosting,0.883594,0.528982
8,adaboost,0.814306,0.679648
9,mlp,0.851753,0.611612


## XGBOOST

## Hyperparameter tuning

In [29]:
from sklearn.model_selection import RandomizedSearchCV

In [47]:
param_grid = {
    'regressor__n_estimators': [100, 200, 300,400,450],
    'regressor__max_depth': [3, 5, 7, 10],
    'regressor__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'regressor__subsample': [0.6, 0.8, 1.0],
    'regressor__colsample_bytree': [0.6, 0.8, 1.0],
    'regressor__gamma': [0, 1, 5],
    'regressor__reg_alpha': [0, 0.1, 1],  # L1 regularization
    'regressor__reg_lambda': [1, 1.5, 2]  # L2 regularization
}


In [49]:
columns_to_encode = ['property_type', 'balcony', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [51]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

In [161]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [163]:
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=100,              # Try 50 random combinations (you can change this)
    cv=kfold,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
    verbose=4
)

In [165]:
search.fit(X, y_transformed)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


,estimator,"Pipeline(step...=None, ...))])"
,param_distributions,"{'regressor__colsample_bytree': [0.6, 0.8, ...], 'regressor__gamma': [0, 1, ...], 'regressor__learning_rate': [0.01, 0.05, ...], 'regressor__max_depth': [3, 5, ...], ...}"
,n_iter,100
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,4
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [167]:
search.best_params_

{'regressor__subsample': 1.0,
 'regressor__reg_lambda': 2,
 'regressor__reg_alpha': 0.1,
 'regressor__n_estimators': 300,
 'regressor__max_depth': 10,
 'regressor__learning_rate': 0.05,
 'regressor__gamma': 0,
 'regressor__colsample_bytree': 0.6}

In [169]:
search.best_score_

0.9061992869366599

In [171]:
y_pred_log = search.predict(X)           # Predicted values (still in log scale if y was log-transformed)
y_pred = np.expm1(y_pred_log)


In [173]:
mae =mean_absolute_error(y, y_pred)

In [175]:
mae

0.11356868589383938

## Exporting the model

In [179]:
columns_to_encode = ['property_type', 'balcony', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [183]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        subsample=1.0,
        reg_lambda=2,
        reg_alpha=0.1,
        n_estimators=300,
        max_depth=10,
        learning_rate=0.05,
        gamma=0,
        colsample_bytree=0.6,
        random_state=42
    ))
])

In [185]:
pipeline.fit(X,y_transformed)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [203]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='neg_mean_absolute_error')

In [205]:
scores.mean()

-0.11219164557297598

In [207]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [209]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)